In [ ]:
!pip install signate

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth


auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'signate.json'", fields="files(id)").execute()
signate_api_key = results.get('files', [])

filename = "/root/.signate/signate.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=signate_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

     |████████████████████████████████| 138 kB 8.5 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=172ed75bd859d6e7b9dac04ca60d29747b6685f0cddb47881f04f9c5e1cc437d
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


Download 100%.


In [ ]:
! signate list 
! signate files --competition-id=406
! signate download --competition-id=406

  competitionId  title                                                                  closing     prize        submitters
---------------  ---------------------------------------------------------------------  ----------  ---------  ------------
              1  【練習問題】銀行の顧客ターゲティング                                   -                              5236
             24  【練習問題】お弁当の需要予測                                           -                              6448
             27  【練習問題】Jリーグの観客動員数予測                                    -                              1445
            100  【練習問題】手書き文字認識                                             -           Knowledge           189
            102  【練習問題】タイタニックの生存予測                                     -           Knowledge          1132
            103  【練習問題】音楽ラベリング                                             -           Knowledge            61
            104  【練習問題】スパムメール分類                                           -           Knowledge     

In [ ]:
! unzip test_data.zip

Archive:  test_data.zip
   creating: test_data/
  inflating: test_data/789907.jpeg   
   creating: __MACOSX/
   creating: __MACOSX/test_data/
  inflating: __MACOSX/test_data/._789907.jpeg  
  inflating: test_data/439948.jpeg   
  inflating: __MACOSX/test_data/._439948.jpeg  
  inflating: test_data/489913.jpeg   
  inflating: __MACOSX/test_data/._489913.jpeg  
  inflating: test_data/429933.jpeg   
  inflating: __MACOSX/test_data/._429933.jpeg  
  inflating: test_data/499929.jpeg   
  inflating: __MACOSX/test_data/._499929.jpeg  
  inflating: test_data/259831.jpeg   
  inflating: __MACOSX/test_data/._259831.jpeg  
  inflating: test_data/649971.jpeg   
  inflating: __MACOSX/test_data/._649971.jpeg  
  inflating: test_data/729911.jpeg   
  inflating: __MACOSX/test_data/._729911.jpeg  
  inflating: test_data/1009937.jpeg  
  inflating: __MACOSX/test_data/._1009937.jpeg  
  inflating: test_data/249889.jpeg   
  inflating: __MACOSX/test_data/._249889.jpeg  
  inflating: test_data/299953.jpeg 

In [ ]:
# ディレクトリ内のデータの確認
%ls

adc.json  __MACOSX/     sample_submission.csv  test_data.zip  train_data.zip
drive/    sample_data/  test_data/             train.csv


In [ ]:
# Googleドライブのマウント
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# ライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os

import torch
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset
from torchvision.io import read_image
from torch.utils.data import DataLoader
from torch import nn, optim

'''
定数の指定
'''

# 学習済みパラメータのパス
model_path = '/content/drive/My Drive/Colab Notebooks/SIGNATE/casting_pytorch/model/casting_pytorch_Epoch17_logloss_0.0002.pth'

# 提出データの保存先
submit_path = '/content/drive/My Drive/Colab Notebooks/SIGNATE/casting_pytorch/submit/casting_pytorch_mobilenet_v3_ver1.csv'

# 学習データのラベルマスター
test_labels = './sample_submission.csv'
sample_submit = './sample_submission.csv'

# 画像データのディレクトリ
img_dir = './test_data/'

# リサイズする画像サイズ
photo_size = 300

# クラス数の定義
num_classes = 2

# 学習に使用する機器(device)の設定
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'デバイス：{device}')

'''
学習済みパラメータの読み込み
'''

model = models.mobilenet_v3_large(pretrained=False)
fc_in_features = model.classifier[3].in_features # 最終レイヤー関数の次元数
model.fc = nn.Linear(fc_in_features, num_classes) # 最終レイヤー関数の付け替え

# モデルをGPUに送る
model.to(device)

# 学習済みパラメータの読み込み
trained_params = torch.load(model_path)

# モデルにパラメータをロード
model.load_state_dict(trained_params)

'''
データの読み込み
'''

# ラベルデータの読み込み
test_labels = pd.read_csv(test_labels, header=None, sep=',')
print(test_labels.head())

# 画像データの名前リストの抽出
x_test = test_labels[0].values
dummy = test_labels[0].values

'''
前処理とデータセットの作成
'''

# transformの設定
transform = {
    'train': transforms.Compose([
        transforms.Resize(photo_size),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]),
    'val': transforms.Compose([
        transforms.Resize(photo_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]),
}

# Datasetの設定
class CastingDataset(Dataset):
    def __init__(self, image_name_list, label_list, img_dir, phase=None):
        self.image_name_list = image_name_list # 画像ファイル名
        self.label_list = label_list # ラベル
        self.img_dir = img_dir # 画像データのディレクトリ
        self.phase = phase # 変数phaseで学習(train)もしくは検証(val)の設定を行う
        self.transform = transform

    def __len__(self):
        return len(self.image_name_list) # 1エポックあたりに読み込むデータ数として、入力データの数を指定

    def __getitem__(self, index):
        
        # index番目の画像を読み込み、前処理を行う
        image_path = os.path.join(self.img_dir, self.image_name_list[index]) # train_master.iloc[index, 0]はファイル名を抽出
        img = Image.open(image_path)
        img = self.transform[self.phase](img)
        
        # index番目のラベルを取得する
        label = self.label_list[index]
        
        return img, label

# Datasetのインスタンス作成
test_dataset = CastingDataset(x_test, dummy, img_dir, phase='val')

# Dataloader
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

'''
テストデータの予測
'''

# 予測データフレームの作成
preds = []

# dataloaderから、ミニバッチ単位でデータを読み込む
for images, _ in test_dataloader:
    
    # 入力データをdeviceへ
    images = images.to(device)
    
    # 学習済みモデルを推論モードに設定
    model.eval()
    
    # モデルによる変換
    outputs = model(images)
    pred = torch.argmax(outputs, dim=1)
    pred = pred.to('cpu').numpy()

    # 予測値をリストに追加
    preds.extend(pred)

'''
提出
'''

# 提出用データの読み込み
sub = pd.read_csv(sample_submit, header=None, sep=',')
print(sub.head())

# 目的変数カラムの置き換え
sub[1] = preds

# ファイルのエクスポート
sub.to_csv(submit_path, sep=',', header=None, index=None)

デバイス：cuda
            0  1
0  19871.jpeg  0
1  29934.jpeg  0
2  39826.jpeg  0
3  49888.jpeg  0
4  59918.jpeg  0
            0  1
0  19871.jpeg  0
1  29934.jpeg  0
2  39826.jpeg  0
3  49888.jpeg  0
4  59918.jpeg  0
